In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys

sys.path.append('../')
sys.path.append('../../')
from DataBot.data_bot import BotMLP
from Agents.ReinforceAgent import Agent
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = Agent(17, 4)
bot0 = Bot()

wins = []

output_dir = '../model_output/REINFORCE_v6/0001/weights_0400.hdf5'

agent0.load_model(output_dir)

for ep in tqdm(range(0, 10000), ascii=True, unit="e"):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2, 2, loc=3, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 0:
                    s_t = env.convert_state(player_turn)
                    action = agent0.act_test(s_t, action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2, 2, loc=3, scale=1)
                s_, reward, game_over, player_turn_temp, _ = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if episode_reward[0] > episode_reward[1]:
        wins.append(1)
    else:
        wins.append(0)

    if ep % 1000 == 0 and ep != 0:
        print(np.sum(wins))

2022-11-01 15:05:41.258771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:05:41.268447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:05:41.269025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 15:05:41.277862: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

598


 20%|##        | 2002/10000 [05:05<20:09,  6.61e/s]

1177


 30%|###       | 3002/10000 [07:36<18:49,  6.19e/s]

1754


 40%|####      | 4002/10000 [10:07<15:18,  6.53e/s]

2315


 46%|####5     | 4593/10000 [11:37<14:55,  6.04e/s]

In [ ]:
np.sum(wins)

5819